In [1]:
import numpy as np
from collections import Counter
from nltk import ngrams

# Unigram Precision

In [2]:
def simple_count(tokens, n):
    return Counter(ngrams(tokens, n))

In [4]:
candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."
tokens = candidate.split() # tokenizing
print(tokens)
print(simple_count(tokens, 1))

['It', 'is', 'a', 'guide', 'to', 'action', 'which', 'ensures', 'that', 'the', 'military', 'always', 'obeys', 'the', 'commands', 'of', 'the', 'party.']
Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


In [6]:
candidate = 'the the the the the the the'
tokens = candidate.split() # 토큰화
print(tokens)
result = simple_count(tokens, 1) # n = 1은 유니그램
print('유니그램 카운트 :',result)

['the', 'the', 'the', 'the', 'the', 'the', 'the']
유니그램 카운트 : Counter({('the',): 7})


# Modified Unigram Precision

In [11]:
def count_clip(candidate, reference_list, n):
    # Ca 문장에서 n-gram 카운트
    ca_cnt = simple_count(candidate, n)
    max_ref_cnt_dict = dict()
    
    for ref in reference_list:
        # Ref 문장에서 n-gram 카운트
        ref_cnt = simple_count(ref, n)

        # 각 Ref 문장에 대해서 비교하여 n-gram의 최대 등장 횟수를 계산.
        for n_gram in ref_cnt:
            if n_gram in max_ref_cnt_dict:
                max_ref_cnt_dict[n_gram] = max(ref_cnt[n_gram], max_ref_cnt_dict[n_gram])
            else:
                max_ref_cnt_dict[n_gram] = ref_cnt[n_gram]

    return {
        # count_clip = min(count, max_ref_count)
        n_gram: min(ca_cnt.get(n_gram, 0), max_ref_cnt_dict.get(n_gram, 0)) for n_gram in ca_cnt
    }

In [8]:
candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat'
]
result = count_clip(candidate.split(),list(map(lambda ref: ref.split(), references)),1)
print('보정된 유니그램 카운트 :',result)

보정된 유니그램 카운트 : {('the',): 2}


In [12]:
def modified_precision(candidate, reference_list, n):
    clip_cnt = count_clip(candidate, reference_list, n)
    total_clip_cnt = sum(clip_cnt.values()) # 분자

    cnt = simple_count(candidate, n)
    total_cnt = sum(cnt.values()) # 분모

    # 분모가 0이 되는 것을 방지
    if total_cnt == 0:
        total_cnt = 1

    # 분자 : count_clip의 합, 분모 : 단순 count의 합 ==> 보정된 정밀도
    return (total_clip_cnt / total_cnt)

# Brevity Penalty

In [13]:
# Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
def closest_ref_length(candidate, reference_list):
    ca_len = len(candidate) # ca 길이
    ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    return closest_ref_len

In [14]:
def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    ref_len = closest_ref_length(candidate, reference_list)

    if ca_len > ref_len:
        return 1

    # candidate가 비어있다면 BP = 0 → BLEU = 0.0
    elif ca_len == 0 :
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

# BLEU Score

In [15]:
def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP

    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights,start=1)]
    # p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

# BLEU Score using NLTK

In [16]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print('실습 코드의 BLEU :',bleu_score(candidate.split(),list(map(lambda ref: ref.split(), references))))
print('패키지 NLTK의 BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

실습 코드의 BLEU : 0.5045666840058485
패키지 NLTK의 BLEU : 0.5045666840058485
